### titanic_train.csv 파일을 로드하고, 이를 DataFrame으로 변환

In [22]:
from pyspark.sql import SparkSession

# SparkSession 객체 생성
spark = SparkSession.builder \
    .appName("stock") \
    .getOrCreate()



# pandas 버전
# pandas_df = pd.read_csv('titanic_train.csv', header='infer')

#spark.read.csv() 메소드를 이용하여 csv 파일을 로드하고 DataFrame으로 변환. 
# CSV 파일을 pandas 데이터프레임으로 불러올 때, 'NULL' 문자열을 NaN 값으로 대체하도록 설정
stock_sdf = spark.read.csv('./data/stock_info_0315.csv', header=True, inferSchema=True,  nullValue='NULL')
print('stock sdf type:', type(stock_sdf))

stock_sdf.show(5)

# spark DataFrame을 메모리에 cache
stock_sdf = stock_sdf.cache()

stock sdf type: <class 'pyspark.sql.dataframe.DataFrame'>
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|종목코드|      종목명| 종가|등락률|    시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|  거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|  000020|    동화약품| 9350| -0.85|261159244500|2024-03-15| 736.0| 12.7|   null|   null| 13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02| 9450| 9450| 9300| 7220

In [23]:
import pandas as pd

na_values = ['NULL']
stock_pdf = pd.read_csv('./data/stock_info_0315.csv', header='infer', na_values=na_values)

stock_pdf.head(5)

,종목코드,종목명,종가,등락률,시가총액,기준일,eps,per,선행eps,선행per,...,시가,고가,저가,거래량,거래대금,기업고유번호,시장구분,종목구분,섹터,업종명
0,000020,동화약품,9350,-0.85,261159244500,2024-03-15,736.0,12.70,NaN,NaN,...,9450,9450,9300,72206,674278570,119195.0,KOSPI,보통주,건강관리,의약품
1,000040,KR모터스,465,0.00,44704386225,2024-03-15,NaN,NaN,NaN,NaN,...,0,0,0,0,0,112378.0,KOSPI,보통주,경기관련소비재,운수장비
2,000050,경방,8470,-1.63,232207336900,2024-03-15,177.0,47.85,NaN,NaN,...,8610,8690,8270,11024,93485040,101628.0,KOSPI,보통주,경기관련소비재,유통업
3,000070,삼양홀딩스,71300,-1.93,610632522300,2024-03-15,9173.0,7.77,NaN,NaN,...,73000,74500,71300,32624,2341408000,126937.0,KOSPI,보통주,소재,기타금융
4,000075,삼양홀딩스우,54900,-0.18,16692784200,2024-03-15,NaN,NaN,NaN,NaN,...,55400,55400,54800,96,5264000,NaN,KOSPI,우선주,NaN,기타금융


In [41]:

import pandas as pd

# pandas DataFrame을 spark DataFrame으로 부터 생성. 
stock_pdf = stock_sdf.select('*').toPandas()
print(type(stock_pdf))

# pandas DataFrame은 spark DataFrame으로 변환
stock_sdf = spark.createDataFrame(stock_pdf)
print(type(stock_sdf))

<class 'pandas.core.frame.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [15]:
# spark DataFrame을 메모리에 cache
titianic_sdf = titanic_sdf.cache()

#### pandas DataFrame의 head()와 spark DataFrame head() 비교 및 print() 적용 시 차이
* pandas DataFrame의 head(N)는 Dataframe의 선두 N개 Record를 가지는 DataFrame을 반환
* spark DataFrame의 head(N)는 DataFrame의 선두 N개 Row Object를 list로 반환. 
* spark DataFrame의 limit(N)가 DataFrame의 선두 N개 Record를 가지는 DataFrame을 반환. 
* pandas DataFrame은 print() 적용 시 DataFrame의 내용을 출력하지만 spark DataFrame은 DataFrame의 schema만 출력

In [11]:
# pandas DataFrame.head(N)는 선두 N개 Record를 가지는 DataFrame 반환
print(type(stock_pdf.head(10)))
# Pandas DataFrame은 print()로 DataFrame의 내용을 출력. 

<class 'pandas.core.frame.DataFrame'>


In [12]:
# spark DataFrame의 head(N)는 DataFrame의 선두 N개 Row Object를 list로 반환.
print(type(stock_sdf.head(10)))
print(stock_sdf.head(10))

<class 'list'>
[Row(종목코드='000020', 종목명='동화약품', 종가=9350, 등락률=-0.85, 시가총액=261159244500, 기준일=datetime.date(2024, 3, 15), eps='736.0', per='12.7', 선행eps='NULL', 선행per='NULL', bps='13165.0', pbr='0.71', 주당배당금='180.0', 배당수익률='1.93', 외국인보유수량=1681856, 외국인지분율=6.02, 외국인한도수량=27931470, 외국인한도소진율=6.02, 시가=9450, 고가=9450, 저가=9300, 거래량=72206, 거래대금=674278570, 기업고유번호='00119195', 시장구분='KOSPI', 종목구분='보통주', 섹터='건강관리', 업종명='의약품'), Row(종목코드='000040', 종목명='KR모터스', 종가=465, 등락률=0.0, 시가총액=44704386225, 기준일=datetime.date(2024, 3, 15), eps='NULL', per='NULL', 선행eps='NULL', 선행per='NULL', bps='345.0', pbr='1.35', 주당배당금='0.0', 배당수익률='0.0', 외국인보유수량=43963412, 외국인지분율=45.73, 외국인한도수량=96138465, 외국인한도소진율=45.73, 시가=0, 고가=0, 저가=0, 거래량=0, 거래대금=0, 기업고유번호='00112378', 시장구분='KOSPI', 종목구분='보통주', 섹터='경기관련소비재', 업종명='운수장비'), Row(종목코드='000050', 종목명='경방', 종가=8470, 등락률=-1.63, 시가총액=232207336900, 기준일=datetime.date(2024, 3, 15), eps='177.0', per='47.85', 선행eps='NULL', 선행per='NULL', bps='30304.0', pbr='0.28', 주당배당금='125.0', 배당수익률='1.48', 외국인보유

In [13]:
stock_sdf

DataFrame[종목코드: string, 종목명: string, 종가: int, 등락률: double, 시가총액: bigint, 기준일: date, eps: string, per: string, 선행eps: string, 선행per: string, bps: string, pbr: string, 주당배당금: string, 배당수익률: string, 외국인보유수량: bigint, 외국인지분율: double, 외국인한도수량: bigint, 외국인한도소진율: double, 시가: int, 고가: int, 저가: int, 거래량: int, 거래대금: bigint, 기업고유번호: string, 시장구분: string, 종목구분: string, 섹터: string, 업종명: string]

In [15]:
# spark DataFrame의 limit(N)은 DataFrame의 선두 N개 Record를 가지는 DataFrame을 반환.
print(type(stock_sdf.limit(10)))

# spark DataFrame은 pandas와 다르게 print() 호출 시 값을 출력하지 않고 DataFrame의 schema만 출력.  
print(stock_sdf.limit(10))

<class 'pyspark.sql.dataframe.DataFrame'>
DataFrame[종목코드: string, 종목명: string, 종가: int, 등락률: double, 시가총액: bigint, 기준일: date, eps: string, per: string, 선행eps: string, 선행per: string, bps: string, pbr: string, 주당배당금: string, 배당수익률: string, 외국인보유수량: bigint, 외국인지분율: double, 외국인한도수량: bigint, 외국인한도소진율: double, 시가: int, 고가: int, 저가: int, 거래량: int, 거래대금: bigint, 기업고유번호: string, 시장구분: string, 종목구분: string, 섹터: string, 업종명: string]


In [25]:
# spark DataFrame의 내용을 출력하기 위해서는 show() 메소드를 사용해야 함. 
stock_sdf.limit(5).show()

+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|종목코드|      종목명| 종가|등락률|    시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|  거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|  000020|    동화약품| 9350| -0.85|261159244500|2024-03-15| 736.0| 12.7|   null|   null| 13165.0|0.71|     180.0|      1.93|       1681856|        6.02|      27931470|            6.02| 9450| 9450| 9300| 72206| 674278570|      119195|   KOSPI|  보통주|      건강관리|  의약품|

In [16]:
# Databricks의 display()는 spark DataFrame의 내용을 출력할 수 있음. 
# display(titanic_sdf)

# 메모리 절약을 위해 limit() 사용이 바람직 
display(stock_sdf.limit(10))

DataFrame[종목코드: string, 종목명: string, 종가: int, 등락률: double, 시가총액: bigint, 기준일: date, eps: string, per: string, 선행eps: string, 선행per: string, bps: string, pbr: string, 주당배당금: string, 배당수익률: string, 외국인보유수량: bigint, 외국인지분율: double, 외국인한도수량: bigint, 외국인한도소진율: double, 시가: int, 고가: int, 저가: int, 거래량: int, 거래대금: bigint, 기업고유번호: string, 시장구분: string, 종목구분: string, 섹터: string, 업종명: string]

### pandas DataFrame의 info()에 대응하는 spark DataFrame 메소드와 로직.
* pandas DataFrame의 info()는 컬럼명, data type과 not null 건수를 함께 출력
* spark DataFrame의 info() 메소드가 없으며 대신 printSchema() 메소드로 스키마(컬럼명, data type)만 출력
* not null 건수를 위해서는 별도의 SQL성 쿼리 작성 필요.

In [26]:
stock_pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2671 entries, 0 to 2670
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   종목코드      2671 non-null   object 
 1   종목명       2671 non-null   object 
 2   종가        2671 non-null   int32  
 3   등락률       2671 non-null   float64
 4   시가총액      2671 non-null   int64  
 5   기준일       2671 non-null   object 
 6   eps       1600 non-null   float64
 7   per       1600 non-null   float64
 8   선행eps     511 non-null    float64
 9   선행per     511 non-null    float64
 10  bps       2381 non-null   float64
 11  pbr       2381 non-null   float64
 12  주당배당금     2622 non-null   float64
 13  배당수익률     2622 non-null   float64
 14  외국인보유수량   2671 non-null   int64  
 15  외국인지분율    2671 non-null   float64
 16  외국인한도수량   2671 non-null   int64  
 17  외국인한도소진율  2671 non-null   float64
 18  시가        2671 non-null   int32  
 19  고가        2671 non-null   int32  
 20  저가        2671 non-null   int3

In [27]:
stock_sdf.printSchema()

root
 |-- 종목코드: string (nullable = true)
 |-- 종목명: string (nullable = true)
 |-- 종가: integer (nullable = true)
 |-- 등락률: double (nullable = true)
 |-- 시가총액: long (nullable = true)
 |-- 기준일: date (nullable = true)
 |-- eps: double (nullable = true)
 |-- per: double (nullable = true)
 |-- 선행eps: double (nullable = true)
 |-- 선행per: double (nullable = true)
 |-- bps: double (nullable = true)
 |-- pbr: double (nullable = true)
 |-- 주당배당금: double (nullable = true)
 |-- 배당수익률: double (nullable = true)
 |-- 외국인보유수량: long (nullable = true)
 |-- 외국인지분율: double (nullable = true)
 |-- 외국인한도수량: long (nullable = true)
 |-- 외국인한도소진율: double (nullable = true)
 |-- 시가: integer (nullable = true)
 |-- 고가: integer (nullable = true)
 |-- 저가: integer (nullable = true)
 |-- 거래량: integer (nullable = true)
 |-- 거래대금: long (nullable = true)
 |-- 기업고유번호: integer (nullable = true)
 |-- 시장구분: string (nullable = true)
 |-- 종목구분: string (nullable = true)
 |-- 섹터: string (nullable = true)
 |-- 업종명: string (nullable 

In [31]:
# SQL의 count case when과 유사한 로직으로 null 값을 포함한 컬럼 확인하기 
from pyspark.sql.functions import count, isnan, when, col

# '기준일' 컬럼을 제외하고 NULL 값 확인
stock_sdf.select([count(when((isnan(c) | col(c).isNull()), c)).alias(c) for c in stock_sdf.columns if c != '기준일']).show()

+--------+------+----+------+--------+----+----+-------+-------+---+---+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+--------+------------+--------+--------+----+------+
|종목코드|종목명|종가|등락률|시가총액| eps| per|선행eps|선행per|bps|pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율|시가|고가|저가|거래량|거래대금|기업고유번호|시장구분|종목구분|섹터|업종명|
+--------+------+----+------+--------+----+----+-------+-------+---+---+----------+----------+--------------+------------+--------------+----------------+----+----+----+------+--------+------------+--------+--------+----+------+
|       0|     0|   0|     0|       0|1071|1071|   2160|   2160|290|290|        49|        49|             0|           0|             0|               0|   0|   0|   0|     0|       0|         117|       0|       0| 395|     0|
+--------+------+----+------+--------+----+----+-------+-------+---+---+----------+----------+--------------+------------+--------------+----------------+----+----+----+-----

### pandas DataFrame describe()와 spark DataFrame describe() 비교
* spark dataframe은 pandas dataframe과 비슷하게 describe()를 통해 모든 컬럼값들의 건수/평균/표준편차/최소값/최대값 등의 정보를 확인할 수 있음. 다만 percentile값을 만들지 않음. 
* pandas dataframe과 다르게 describe()시 숫자형 컬럼 뿐만 아니라 문자형 컬럼에 대해서도 건수/평균/표준편차/최소값/최대값 조사

In [32]:
stock_pdf.describe()

,종가,등락률,시가총액,eps,per,선행eps,선행per,bps,pbr,주당배당금,...,외국인보유수량,외국인지분율,외국인한도수량,외국인한도소진율,시가,고가,저가,거래량,거래대금,기업고유번호
count,2671.000000,2671.000000,2.671000e+03,1600.000000,1600.000000,511.000000,511.000000,2.381000e+03,2381.000000,2622.000000,...,2.671000e+03,2671.000000,2.671000e+03,2671.000000,2671.000000,2671.000000,2671.000000,2.671000e+03,2.671000e+03,2.554000e+03
mean,21148.824410,-0.365279,9.689956e+11,2816.378125,51.476894,4879.381605,24.679687,2.418203e+04,2.581033,343.768879,...,5.286946e+06,5.912029,4.268958e+07,6.012894,21096.535006,21493.001872,20686.351554,6.333656e+05,8.574878e+09,5.986314e+05
std,52048.083095,2.925279,9.389447e+12,12860.281121,338.576691,7325.466240,58.653900,1.262079e+05,12.185354,1160.520751,...,6.651611e+07,10.774651,1.359081e+08,11.103359,52288.363394,53247.629079,51425.813458,3.343919e+06,4.485740e+10,4.838963e+05
min,77.000000,-29.980000,3.530573e+09,1.000000,0.210000,18.000000,2.090000,3.400000e+01,0.030000,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,1.006010e+05
25%,2762.500000,-1.510000,5.962481e+10,200.750000,7.000000,874.000000,7.070000,2.123000e+03,0.580000,0.000000,...,1.029495e+05,0.650000,1.090727e+07,0.650000,2567.500000,2615.000000,2522.500000,2.134350e+04,1.077812e+08,1.613938e+05
50%,6390.000000,-0.300000,1.214701e+11,624.500000,13.205000,2138.000000,10.830000,5.511000e+03,1.070000,0.000000,...,3.847710e+05,1.740000,2.062685e+07,1.750000,6260.000000,6320.000000,6170.000000,8.436400e+04,4.698871e+08,4.156370e+05
75%,16745.000000,0.415000,3.049408e+11,1966.750000,29.270000,5528.500000,19.045000,1.384900e+04,2.160000,200.000000,...,1.601578e+06,6.015000,4.230224e+07,6.040000,16660.000000,17040.000000,16275.000000,2.805350e+05,2.615731e+09,9.922318e+05
max,826000.000000,29.950000,4.316153e+14,406090.000000,8700.000000,58014.000000,762.780000,4.906107e+06,522.350000,20000.000000,...,3.254505e+09,83.230000,5.969783e+09,92.280000,837000.000000,855000.000000,826000.000000,1.016378e+08,1.646273e+12,1.806951e+06


In [33]:
# spark DataFrame의 describe() 역시 DataFrame을 반환하므로 내용 출력을 위해서는 show() 메소드를 호출하거나 Databricks의 display()로 감싸야 함. 
stock_sdf.describe().show()

+-------+------------------+------+------------------+-------------------+--------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+--------------------+-----------------+--------+--------+----------+------+
|summary|          종목코드|종목명|              종가|             등락률|            시가총액|               eps|               per|          선행eps|           선행per|               bps|               pbr|        주당배당금|        배당수익률|     외국인보유수량|      외국인지분율|     외국인한도수량|  외국인한도소진율|              시가|             고가|              저가|            거래량|            거래대금|     기업고유번호|시장구분|종목구분|      섹터|업종명|
+-------+------------------+------+------------------+-------------------+--------------------+------------------+------------------+-------

In [34]:
stock_sdf.dtypes

[('종목코드', 'string'),
 ('종목명', 'string'),
 ('종가', 'int'),
 ('등락률', 'double'),
 ('시가총액', 'bigint'),
 ('기준일', 'date'),
 ('eps', 'double'),
 ('per', 'double'),
 ('선행eps', 'double'),
 ('선행per', 'double'),
 ('bps', 'double'),
 ('pbr', 'double'),
 ('주당배당금', 'double'),
 ('배당수익률', 'double'),
 ('외국인보유수량', 'bigint'),
 ('외국인지분율', 'double'),
 ('외국인한도수량', 'bigint'),
 ('외국인한도소진율', 'double'),
 ('시가', 'int'),
 ('고가', 'int'),
 ('저가', 'int'),
 ('거래량', 'int'),
 ('거래대금', 'bigint'),
 ('기업고유번호', 'int'),
 ('시장구분', 'string'),
 ('종목구분', 'string'),
 ('섹터', 'string'),
 ('업종명', 'string')]

In [35]:
# number형 컬럼들에 대해서만 describe()수행할 수 있도록 select 컬럼 filtering 적용. 
number_columns = [column_name for column_name, dtype in stock_sdf.dtypes if dtype != 'string']
print(number_columns)

stock_sdf.select(number_columns).describe().show()

['종가', '등락률', '시가총액', '기준일', 'eps', 'per', '선행eps', '선행per', 'bps', 'pbr', '주당배당금', '배당수익률', '외국인보유수량', '외국인지분율', '외국인한도수량', '외국인한도소진율', '시가', '고가', '저가', '거래량', '거래대금', '기업고유번호']
+-------+------------------+-------------------+--------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-----------------+------------------+------------------+--------------------+-----------------+
|summary|              종가|             등락률|            시가총액|               eps|               per|          선행eps|           선행per|               bps|               pbr|        주당배당금|        배당수익률|     외국인보유수량|      외국인지분율|     외국인한도수량|  외국인한도소진율|              시가|             고가|              저가|            거래량|            거래대금|     기업고유번호|
+-------+------------------+-------------------+----------------

In [36]:
stock_pdf.shape

(2671, 28)

### pandas DataFrame의 shape에 이에 대응하는 spark Dataframe 로직
* pandas DataFrame의 shape는 row건수와 column 개수를 매우 빠르게 제공.
* spark DataFrame은 shape를 제공하지 않음. column 개수는 spark DataFrame의 columns로 확인. row건수는 count() 메소드(쿼리성)로 확인.

In [37]:
# spark DataFrame은 columns 속성으로 컬럼명을 list로 반환. 
print('column들:', stock_sdf.columns)
print('column개수:', len(stock_sdf.columns))

column들: ['종목코드', '종목명', '종가', '등락률', '시가총액', '기준일', 'eps', 'per', '선행eps', '선행per', 'bps', 'pbr', '주당배당금', '배당수익률', '외국인보유수량', '외국인지분율', '외국인한도수량', '외국인한도소진율', '시가', '고가', '저가', '거래량', '거래대금', '기업고유번호', '시장구분', '종목구분', '섹터', '업종명']
column개수: 28


In [38]:
# spark DataFrame은 count() 메소드로 전체 row 건수를 제공. 
print(stock_sdf.count())
print(type(stock_sdf.count()))

2671
<class 'int'>


In [39]:
print('stock_sdf shape:', (titanic_sdf.count(), len(titanic_sdf.columns)))

stock_sdf shape: (2671, 28)


### Spark DataFrame의 select() 메소드 알아보기
* select() 메소드는 SQL의 Select 절과 유사하게 한개 이상의 컬럼들의 값을 DataFrame형태로 반환. 
* 한개의 컬럼명, 또는 여러개의 컬럼명을 인자로 입력할 수 있음.
* 개별 컬럼명을 문자열 형태로 또는 DataFrame의 컬럼 속성으로 지정
* DataFrame의 컬럼 속성으로 지정시에는 DataFrame.컬럼명, DataFrame[컬럼명], col(컬럼명) 으로 지정 가능.

In [43]:
print('pandas DataFrame의 단일 컬럼값을 출력')
print(stock_pdf['종목명'])

print('\npandas DataFrame의 여러개 컬럼값들을 출력')
print(stock_pdf[['종목명', '종목코드']])

pandas DataFrame의 단일 컬럼값을 출력
0             동화약품
1            KR모터스
2               경방
3            삼양홀딩스
4           삼양홀딩스우
           ...    
2666           JTC
2667       고스트스튜디오
2668           소마젠
2669    프레스티지바이오파마
2670         네오이뮨텍
Name: 종목명, Length: 2671, dtype: object

pandas DataFrame의 여러개 컬럼값들을 출력
             종목명    종목코드
0           동화약품  000020
1          KR모터스  000040
2             경방  000050
3          삼양홀딩스  000070
4         삼양홀딩스우  000075
...          ...     ...
2666         JTC  950170
2667     고스트스튜디오  950190
2668         소마젠  950200
2669  프레스티지바이오파마  950210
2670       네오이뮨텍  950220

[2671 rows x 2 columns]


In [44]:
# select() 메소드는 DataFrame을 반환. Name 컬럼을 가지는 DataFrame을 반환. 
stock_sdf.select('종목명').show() # select name from stock_sdf;

+------------------+
|            종목명|
+------------------+
|          동화약품|
|          KR모터스|
|              경방|
|        삼양홀딩스|
|      삼양홀딩스우|
|        하이트진로|
|    하이트진로2우B|
|          유한양행|
|        유한양행우|
|        CJ대한통운|
|  하이트진로홀딩스|
|하이트진로홀딩스우|
|              두산|
|            두산우|
|          두산2우B|
|      성창기업지주|
|                DL|
|              DL우|
|          유유제약|
|       유유제약1우|
+------------------+
only showing top 20 rows



In [47]:
# 여러개의 컬럼명을 select 시에도 개별 컬럼명을 인자로 넣어줌. pandas와 다르게 list를 사용하지 않음. 
stock_sdf.select('종목명', '종가').show(5) # select 종목명, 종가 from data_sdf

# 모든 컬럼을 선택시 SQL과 유사하게 '*' 사용 가능. 
stock_sdf.select('*').show(5) # select * from stock_sdf

+------------+-----+
|      종목명| 종가|
+------------+-----+
|    동화약품| 9350|
|    KR모터스|  465|
|        경방| 8470|
|  삼양홀딩스|71300|
|삼양홀딩스우|54900|
+------------+-----+
only showing top 5 rows

+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|종목코드|      종목명| 종가|등락률|    시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|  거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|  000020|    동화약품| 9350| -0.85|261159244500|2024-03-15| 736.0| 12.7|    

In [48]:
select_columns = stock_sdf.columns

# * list 는 list내부의 원소를 unpack
print('select_columns:', select_columns, *select_columns)

stock_sdf.select(*select_columns).show(5)
#버전 upgrade가 되면서 select 절에 list를 입력해도 여러개의 컬럼을 반환 가능. 하지만 원칙적으로 개별 컬럼들을 인자로 입력하는 것이 바람직. 
stock_sdf.select(select_columns).show(5)

select_columns.remove('종목명')
stock_sdf.select(*select_columns).show(5)

select_columns: ['종목코드', '종목명', '종가', '등락률', '시가총액', '기준일', 'eps', 'per', '선행eps', '선행per', 'bps', 'pbr', '주당배당금', '배당수익률', '외국인보유수량', '외국인지분율', '외국인한도수량', '외국인한도소진율', '시가', '고가', '저가', '거래량', '거래대금', '기업고유번호', '시장구분', '종목구분', '섹터', '업종명'] 종목코드 종목명 종가 등락률 시가총액 기준일 eps per 선행eps 선행per bps pbr 주당배당금 배당수익률 외국인보유수량 외국인지분율 외국인한도수량 외국인한도소진율 시가 고가 저가 거래량 거래대금 기업고유번호 시장구분 종목구분 섹터 업종명
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+--------------+----------------+-----+-----+-----+------+----------+------------+--------+--------+--------------+--------+
|종목코드|      종목명| 종가|등락률|    시가총액|    기준일|   eps|  per|선행eps|선행per|     bps| pbr|주당배당금|배당수익률|외국인보유수량|외국인지분율|외국인한도수량|외국인한도소진율| 시가| 고가| 저가|거래량|  거래대금|기업고유번호|시장구분|종목구분|          섹터|  업종명|
+--------+------------+-----+------+------------+----------+------+-----+-------+-------+--------+----+----------+----------+--------------+------------+-------

In [57]:
# 컬럼 속성으로 지정하여 select 의 인자로 사용 가능. pandas DataFrame의 ['column']는 컬럼값을 포함한 array를 지칭하나 spark DataFrame의 ['column']는 컬럼 자체를 지정함. 
stock_sdf.select(stock_sdf.종목명, stock_sdf.종목코드).show(3)
stock_sdf.select(stock_sdf['종목명'], stock_sdf['종목코드']).show(3)


from pyspark.sql.functions import col

# pyspark.sql.functions의 col() 함수를 이용하여 명시적으로 컬럼명을 지정할 수 있음. 
stock_sdf.select(col('종목명'), col('종목코드')).show(3)

+--------+--------+
|  종목명|종목코드|
+--------+--------+
|동화약품|  000020|
|KR모터스|  000040|
|    경방|  000050|
+--------+--------+
only showing top 3 rows

+--------+--------+
|  종목명|종목코드|
+--------+--------+
|동화약품|  000020|
|KR모터스|  000040|
|    경방|  000050|
+--------+--------+
only showing top 3 rows

+--------+--------+
|  종목명|종목코드|
+--------+--------+
|동화약품|  000020|
|KR모터스|  000040|
|    경방|  000050|
+--------+--------+
only showing top 3 rows



In [58]:
stock_pdf.종목명

0             동화약품
1            KR모터스
2               경방
3            삼양홀딩스
4           삼양홀딩스우
           ...    
2666           JTC
2667       고스트스튜디오
2668           소마젠
2669    프레스티지바이오파마
2670         네오이뮨텍
Name: 종목명, Length: 2671, dtype: object

In [60]:
print(type(stock_sdf.종목명), type(stock_sdf['종목명']))
print(type(col('종목명')))

<class 'pyspark.sql.column.Column'> <class 'pyspark.sql.column.Column'>
<class 'pyspark.sql.column.Column'>


In [65]:
from pyspark.sql.functions import upper, lower, col

# select()에서 컬럼 데이터 가공 후 생성 가능. 
# Select 종목코드, 종목명, 시장구분, lower(시장구분) from stock_sdf 
stock_sdf.select("종목코드", "종목명" , "시장구분",lower(col('시장구분'))).show(5) 
# Select lower(시장구분) as Cap_시장구분 from stock_sdf
stock_sdf.select(lower(col('시장구분')).alias('Cap_시장구분')).show(5) 

+--------+------------+--------+---------------+
|종목코드|      종목명|시장구분|lower(시장구분)|
+--------+------------+--------+---------------+
|  000020|    동화약품|   KOSPI|          kospi|
|  000040|    KR모터스|   KOSPI|          kospi|
|  000050|        경방|   KOSPI|          kospi|
|  000070|  삼양홀딩스|   KOSPI|          kospi|
|  000075|삼양홀딩스우|   KOSPI|          kospi|
+--------+------------+--------+---------------+
only showing top 5 rows

+------------+
|Cap_시장구분|
+------------+
|       kospi|
|       kospi|
|       kospi|
|       kospi|
|       kospi|
+------------+
only showing top 5 rows



### spark DataFrame filter() 메소드 알아보기
* filter()는 SQL의 where와 유사하게 DataFrame내의 특정 조건을 만족하는 레코드를 DataFrame으로 반환. 
* filter()내의 조건 컬럼은 컬럼 속성으로 지정 가능. 조건문 자체는 SQL 과 유사한 문자열로 지정 할 수 있음(조건 컬럼은 문자열 지정이 안됨. )
* where() 메소드는 filter()의 alias이며 SQL where와 직관적인 동일성을 간주하기 위해 생성. 
* 복합 조건 and는 & 를, or를 | 를 사용. 개별 조건은 ()로 감싸야 함.

In [ ]:
dict_01 = {'Name': ['Chulmin', 'Wansoo','Myunghyun','Hyunjoo', 'Chulman'],
           'Year': [2011, 2016, 2015, 2015, 2011],
           'Gender': ['Male', 'Male', 'Male', 'Female', 'Male']
          }
# 딕셔너리를 DataFrame으로 변환
data_pdf = pd.DataFrame(dict_01)

# pandas DataFrame은 spark DataFrame으로 변환
data_sdf = spark.createDataFrame(data_pdf)

print(type(data_sdf))
display(data_sdf)

<class 'pyspark.sql.dataframe.DataFrame'>


Name,Year,Gender
Chulmin,2011,Male
Wansoo,2016,Male
Myunghyun,2015,Male
Hyunjoo,2015,Female
Chulman,2011,Male


In [ ]:
data_sdf.filter(data_sdf['Name'] == 'Chulmin').show() # select * from data_sdf where Name = 'Chulmin'

print('조건 컬럼을 문자열로 지정할 수 없으므로 아래는 오류를 발생 시킵니다. ')
data_sdf.filter('Name' == 'Chulmin').show()

+-------+----+------+
|   Name|Year|Gender|
+-------+----+------+
|Chulmin|2011|  Male|
+-------+----+------+

조건 컬럼을 문자열로 지정할 수 없으므로 아래는 오류를 발생 시킵니다. 


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-3442554736920586> in <module>
      2 
      3 print('조건 컬럼을 문자열로 지정할 수 없으므로 아래는 오류를 발생 시킵니다. ')
----> 4 data_sdf.filter('Name' == 'Chulmin').show()

/databricks/spark/python/pyspark/sql/dataframe.py in filter(self, condition)
   1756             jdf = self._jdf.filter(condition._jc)
   1757         else:
-> 1758             raise TypeError("condition should be string or Column")
   1759         return DataFrame(jdf, self.sql_ctx)
   1760 

TypeError: condition should be string or Column

In [ ]:
data_sdf.where(data_sdf['Name'] == 'Chulmin').show()

+-------+----+------+
|   Name|Year|Gender|
+-------+----+------+
|Chulmin|2011|  Male|
+-------+----+------+



In [ ]:
# 단일 컬럼은 string으로 사용할 수 없으며 filter 절의 조건 자체를 SQL의 where 조건절 구문과 유사하게 string으로 만들어야함. 단 SQL과 다르게 동일 비교 시 = 가 아니라 == 이 사용되어야 함.
# 또한 SQL 내부 문자열은 '' 으로 표시해야함. 
data_sdf.filter("Name == 'Chulmin'").show() # select * from data_sdf where Name = 'Chulmin'

+-------+----+------+
|   Name|Year|Gender|
+-------+----+------+
|Chulmin|2011|  Male|
+-------+----+------+



In [ ]:
data_sdf.filter( (data_sdf['Gender'] == 'Male') & (col('Year') > 2011) ).show() # select * from data_sdf where Gender = 'Male' and Year > 2011
data_sdf.filter( (data_sdf['Gender'] == 'Male') | (col('Year') < 2011) ).show() # select * from data_sdf where Gender = 'Male' or Year < 2011

+---------+----+------+
|     Name|Year|Gender|
+---------+----+------+
|   Wansoo|2016|  Male|
|Myunghyun|2015|  Male|
+---------+----+------+

+---------+----+------+
|     Name|Year|Gender|
+---------+----+------+
|  Chulmin|2011|  Male|
|   Wansoo|2016|  Male|
|Myunghyun|2015|  Male|
|  Chulman|2011|  Male|
+---------+----+------+



In [ ]:
# 문자열 컬럼의 like 조건 수행. 
print('문자열 컬럼 like() filter() 수행. ')
data_sdf.filter(col('Name').like('Chul%')).show() # select * from data_sdf where Name like 'Chul%'

print('SQL의 Like 조건문을 string으로 filter() 수행. ')
data_sdf.filter(" Name like 'Chul%' ").show()
data_sdf.filter(" Name like 'C%' ").show() # startswith('C') where Name like 'C%'
data_sdf.filter(" Name like '%u%' ").show() # contains('u')
data_sdf.filter(" upper(Name) like '%M%' ").show() #별도의 pyspark.sql.functions 호출없이 수행 가능.  select * from data_sdf where upper(Name) like '%M%'

문자열 컬럼 like() filter() 수행. 
SQL의 Like 조건문을 string으로 filter() 수행. 
+-------+----+------+
|   Name|Year|Gender|
+-------+----+------+
|Chulmin|2011|  Male|
|Chulman|2011|  Male|
+-------+----+------+

+-------+----+------+
|   Name|Year|Gender|
+-------+----+------+
|Chulmin|2011|  Male|
|Chulman|2011|  Male|
+-------+----+------+

+---------+----+------+
|     Name|Year|Gender|
+---------+----+------+
|  Chulmin|2011|  Male|
|Myunghyun|2015|  Male|
|  Hyunjoo|2015|Female|
|  Chulman|2011|  Male|
+---------+----+------+

+---------+----+------+
|     Name|Year|Gender|
+---------+----+------+
|  Chulmin|2011|  Male|
|Myunghyun|2015|  Male|
|  Chulman|2011|  Male|
+---------+----+------+



In [ ]:
from pyspark.sql.functions import upper

data_sdf.filter(upper(data_sdf['Name']).like('%M%')).show() #select * from data_sdf where upper(Name) like '%M%'

+---------+----+------+
|     Name|Year|Gender|
+---------+----+------+
|  Chulmin|2011|  Male|
|Myunghyun|2015|  Male|
|  Chulman|2011|  Male|
+---------+----+------+



In [ ]:
# filtering 후에 특정 컬럼으로만 DataFrame 생성하려면 select() 적용. 
data_sdf.filter(upper(data_sdf['Name']).like('%M%')).select('Year', 'Gender').show()

+----+------+
|Year|Gender|
+----+------+
|2011|  Male|
|2015|  Male|
|2011|  Male|
+----+------+

